## Assignment 3 Question 2

Is it possible to predict ELO of a player based on context of a potential en passant move?

Only investigating the player who is in the position of making the en passant move.

Lila See FDS PCA solutions for 3 good points for what PCA is good for - include in report?

#### Import Libraries

In [15]:
import pandas as pd
import chess
import chess.pgn
import chess.engine
import io
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import time

import data_cleaning

# PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#### Import stockfish 🐟🐟🐟

In [16]:
# Windows installation
STOCKFISH_LOC = os.getcwd() + "\stockfish\stockfish-windows-2022-x86-64-avx2.exe"
# Unix based installation i.e. DICE/Codespaces
STOCKFISH_LOC_UNIX = os.getcwd() + "/stockfish_14_linux/stockfish_14_x64"

# Import stockfish package
from stockfish import Stockfish
# Try and read windows location
try:
    fish = Stockfish(path=STOCKFISH_LOC)
# If error, read the linux location
except:
    fish = Stockfish(path=STOCKFISH_LOC_UNIX)
    
# It will throw an error but if it still runs then ignore it :D

Exception ignored in: <function Stockfish.__del__ at 0x7fd7ec1fc0d0>
Traceback (most recent call last):
  File "/afs/inf.ed.ac.uk/user/s22/s2202694/.local/lib/python3.8/site-packages/stockfish/models.py", line 757, in __del__
    if self._stockfish.poll() is None:
AttributeError: 'Stockfish' object has no attribute '_stockfish'


#### Import CSV file

66879 entries in dataframe

In [17]:
chess_data = data_cleaning.import_data()
print(chess_data.dtypes)

white_username    object
black_username    object
white_id          object
black_id          object
white_rating       int64
black_rating       int64
white_result      object
black_result      object
time_class        object
time_control      object
rules             object
rated               bool
fen               object
pgn               object
dtype: object


#### Dataframe info

Using:
print(chess_data.dtypes)
print(chess["rules"].unique())


Chess rules:
['chess' 'chess960' 'threecheck' 'crazyhouse' 'kingofthehill']

Time control:
['1/259200' '1/172800' '1800' '1/86400' '1/432000' '1/604800' '600'
 '120+1' '900+10' '300' '180+2' '3600+5' '2700+45' '3600' '1/1209600'
 '180' '600+10' '60' '480+3' '300+5' '420+3' '600+5' '600+2' '1200' '30'
 '60+1' '120' '1500+3' '900+2' '1500+5' '1500+10' '1/864000' '900' '300+2'
 '1500' '7200' '300+1' '5400' '3600+60' '2700+30' '3480+45' '10' '2700+10'
 '15' '2700' '3600+20' '4500' '4200' '900+5' '1800+10' '2700+5' '480+5'
 '1800+30' '300+3' '600+1' '1800+5' '420+5' '5400+30' '240+10' '420' '303'
 '60+10']

 Time class:
['daily' 'rapid' 'bullet' 'blitz']

#### Clean data

Undeveloped board shouldn't matter if we're filtering games for potential ep

Same for draws

Can filter our time class if are looking at time controls

After making a new move_list column, should we drop the pgn column?


Variables we are considering when predicting ELO (for a player who could potentially make en passant move) are: (Y = DONE, N = NOT DONE)
- Y: Colour who had ep opportunity (boolean)
- Y: Did they take the en passant? (boolean)
- N: Does their choice on taking/not taking support them if gaining an advantage? (numerical value for how much of an advantage it gives)
- N: Time taken to decide to capture/not capture en passant (... whatever can be time, a number in seconds ig)
- Y: Is the game rated? (boolean)
- Y: Game time class


To do:
- Work out which colour is making the potential en passant move, add a column to dataframe detailing this
- Make dataframe columns for other variables
- Apply PCA reduction

In [18]:
# Save PGN column from dataframe
full_pgn = chess_data['pgn']

def get_moves(entry):
    '''
    Retrive series of moves in a game when given the whole full_pgn entry
    '''
    pgn = entry.splitlines()[-1]
    return pgn

# Add list of moves (string) as a new column to dataframe
chess_data['move_list'] = full_pgn.apply(get_moves)
# Drop irrelevant columns
chess_data = chess_data.drop(['time_control', 'white_username', 'black_username','white_id', 'black_id', 'white_result', 'black_result', 'rules'], axis=1)

In [19]:
print(chess_data.iloc[0])

white_rating                                                 1708
black_rating                                                 1608
time_class                                                  daily
rated                                                        True
fen             r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R ...
pgn             [Event "Enjoyable games 2 - Round 1"]\n[Site "...
move_list       1. d4 Nf6 2. c4 e6 3. Nc3 Bb4 4. Qb3 Bxc3+ 5. ...
Name: 0, dtype: object


#### Import chess info
https://python-chess.readthedocs.io/en/latest/core.html#chess.Board.san


#### En Passant functions
- has_legal_en_passant() tests if en passant capturing would actually be possible on the next move.
- has_pseudo_legal_en_passant()
- has_legal_en_passant()
- is_en_passant(move: Move) Checks if the given pseudo-legal move is an en passant capture.




Use StringIO to parse games from a string.

```python
import io
pgn = io.StringIO("1. e4 e5 2. Nf3 *")
game = chess.pgn.read_game(pgn)
```

#### Clean data specifically for ep

Filter out columns that don't have potential ep
Add columns for: whether ep happened, which colour had potential to take ep

In [20]:
def check_pgn(df_row):
    '''
    Checks PGN for whether opportunity for EP happened in the game.
    '''
    
    pgn_file = io.StringIO(df_row["pgn"])   # PGN as a file
    game = chess.pgn.read_game(pgn_file)    # Read PGN and put into game
    board = game.board()                    # "board" of a game
    # pre-initialise df_row values
    precheck = False
    moved = False
    move_colour = ""
    
    # comparison sets when checking move piece
    ep_set = set(["ax","bx","cx","dx","ex","fx","gx", "hx"])
    check_set = set("abcdefgh")
    
    # Find only pawn moves in game
    for move in game.mainline_moves():
        san = board.san(move)
        turn = board.turn
        move_piece = san[0]
        # check if precheck was flagged in previous move
        if precheck:
            # get first 2 letters, and compare to a set
            move_ep_piece = san[:2]
            if move_ep_piece in ep_set:
                # check if e.p. actually happened
                moved = board.is_en_passant(move)
            # break out for loop, no need to check further
            break
        
        # if the move was a pawn (lower case)
        if move_piece in check_set:
            # Push the move before checking the board
            board.push(move)
            # check if the next move can be en passant
            precheck = board.has_legal_en_passant()
            # if precheck is true then set the turn
            if precheck:
                if turn:
                    move_colour = 'Black'
                else:
                    move_colour = 'White'
        # if the move wasn't a pawn then just continue as normal
        else:
            board.push(move)
    
    # set row values
    df_row["ep_opportunity"] = precheck
    df_row["ep_happened"] = moved
    df_row["ep_colour"] = move_colour
    return df_row

chess_filter = chess_data.copy(deep=True)

start = time.time()
# apply e.p. finding function
chess_filter = chess_filter.apply(check_pgn, axis=1)
end = time.time()
print(end - start)

Exception ignored in: <function Stockfish.__del__ at 0x7fd7ec1fc0d0>
Traceback (most recent call last):
  File "/afs/inf.ed.ac.uk/user/s22/s2202694/.local/lib/python3.8/site-packages/stockfish/models.py", line 757, in __del__
    if self._stockfish.poll() is None:
AttributeError: 'Stockfish' object has no attribute '_stockfish'


439.73207998275757


In [21]:
chess_filter = chess_filter[chess_filter['ep_opportunity'] == True]
chess_filter = chess_filter.drop(['ep_opportunity'], axis=1)
chess_filter = chess_filter[chess_filter['time_class'] != "daily"]
chess_filter = chess_filter.reset_index(drop=True)
print(len(chess_filter))
print(len(chess_filter[chess_filter['ep_happened'] == True]))
# print(chess_filter["move_list"].sample(50).head(20))

5074
1563


In [22]:
# print(chess_filter.values)

In [23]:
samp = chess_filter.sample(10)

#### Cleaning data for EP - Advantage for ep

Analysing game - finding if ep move would have given an edvantage


Split into cases: ep happened and ep didn't happen


NOT DONE YET

In [81]:
def increment_letter(letter):
    '''dumb increment function'''
    if letter == "h":
        return "z"
    return chr(ord(letter) + 1)

def decrement_letter(letter):
    '''dumb decrement function'''
    if letter == "a":
        return "y"
    return chr(ord(letter) - 1)

def check_adjacent_piece(square, pawn):
    '''Checks if a piece on a square is an opponent pawn'''
    piece = fish.get_what_is_on_square(square)
    return True if piece == pawn else False
    
def ep_eval(fen, move, col):
    # reset board to pre-move FEN
    fish.set_fen_position(fen)

    # make the move on stockfish
    fish.make_moves_from_current_position([move])
    # print(fish.get_board_visual())
    print("e.p. evaluation: " + str(fish.get_evaluation()["value"]))
    return fish.get_evaluation()["value"]


def get_advantage(df_row):
    # make board
    # find move with e.p.
    # stockfish eval at that point
    # get stockfish eval if best move
    # get stockfish eval if e.p.
    # find difference between two
    # return

    ep_set = set(["ax","bx","cx","dx","ex","fx","gx", "hx"])
    check_set = set("abcdefgh")

    
    pgn_file = io.StringIO(df_row["pgn"])   # PGN as a file
    game = chess.pgn.read_game(pgn_file)    # Read PGN and put into game
    board = game.board()                    # "board" of a game
    precheck = False
    
    pre_eval = 0
    post_eval_best_diff = 0
    post_eval_ep_diff = 0
    
    for move in game.mainline_moves():
        san = board.san(move)
        move_piece = san[0]

        # check if precheck was flagged in previous move
        if precheck:
            col = df_row["ep_colour"]
            # set stockfish on position
            fish.set_fen_position(board.fen())
            
            # get pre-move evaluation
            pre_eval = fish.get_evaluation()
            
            print("pre-evaluation: " + str(pre_eval["value"]))

            # get best move
            best_move = fish.get_best_move()
            print("best move: " + str(best_move))
            # make best move and measure
            fish.make_moves_from_current_position([best_move])
            post_eval = fish.get_evaluation()
            # print(post_evaluation["value"])
            
            print("post-evaluation: " + str(post_eval["value"]))
            
            if col=="White":
                # get difference of best move
                post_eval_best_diff = post_eval["value"] - pre_eval["value"]
            else:
                post_eval_best_diff = -1*(post_eval["value"] - pre_eval["value"])

            # print("board peek: " + str(board.peek()))
            # get previous move
            pawn_move_row = str(board.peek())[-2]
            pawn_move_col = str(board.peek())[-1]
            ep_result_col = "3" if pawn_move_col == "4" else "6"

            # get left and right square relative to pawn that just moved
            left_square = decrement_letter(pawn_move_row) + pawn_move_col
            right_square = increment_letter(pawn_move_row) + pawn_move_col

            # print("leftsq: " + left_square)
            # print("righsq: " + right_square)

            # checks if in leftmost or rightmost file
            left_bound = True if left_square[-2] == "y" else False
            right_bound = True if right_square[-2] == "z" else False
            
            if col == "White":
                FISH_PAWN = fish.Piece.WHITE_PAWN

            else:
                FISH_PAWN = fish.Piece.BLACK_PAWN

            # gross conditional :/
            fish.set_fen_position(board.fen())

            # if the piece to the left of the moved pawn is also a pawn
            ep_left = False
            # if the piece to the right of the moved pawn is also a pawn
            ep_right = False
            
            # if somewhere in the middle
            if (not left_bound) and (not right_bound):
                ep_left = check_adjacent_piece(left_square, FISH_PAWN)
                ep_right = check_adjacent_piece(right_square, FISH_PAWN)
            
            # if A file pawn (leftmost)
            elif left_bound:
                ep_right = check_adjacent_piece(right_square, FISH_PAWN)
            
            # if H file pawn (rightmost)
            elif right_bound:
                ep_left = check_adjacent_piece(left_square, FISH_PAWN)

            # print(ep_left)
            # print(ep_right)
            print(col)
            # low low post_eval value in case there are two diff e.p. states
            # if white then
            
            eval_ep_left = 0
            eval_ep_right = 0
            
            # get eval if left side e.p.
            if ep_left:
                move = decrement_letter(pawn_move_row) + pawn_move_col + pawn_move_row + ep_result_col
                eval_ep_left = ep_eval(board.fen(), move, col)
            
            # get eval right side e.p.
            if ep_right:
                move = increment_letter(pawn_move_row) + pawn_move_col + pawn_move_row + ep_result_col
                eval_ep_right = ep_eval(board.fen(), move, col)
            
            
            best_ep_val = 0
            if (not left_bound) and (not right_bound):
                if col == "White":
                    best_ep_val = max(eval_ep_left, eval_ep_right)
                else:
                    best_ep_val = min(eval_ep_left, eval_ep_right)
            elif left_bound:
                best_ep_val = eval_ep_right
            else:
                best_ep_val = eval_ep_left


            if col == "White":
                print("diff: " + str(best_ep_val))
                post_eval_ep_diff = best_ep_val - pre_eval["value"]
            else:
                print("diff: " + str(best_ep_val))
                post_eval_ep_diff = -1*(best_ep_val - pre_eval["value"])
            break
            
        # if the move was a pawn (lower case)
        if move_piece in check_set:
            # Push the move before checking the board
            board.push(move)
            # check if the next move can be en passant
            precheck = board.has_legal_en_passant()
        # if the move wasn't a pawn then just continue as normal
        else:
            board.push(move)
    
    print(post_eval_ep_diff)
    print()
    df_row["pre_state"] = pre_eval["value"]
    df_row["state_best"] = post_eval_best_diff# stockfish eval if e.p. is taken
    df_row["state_ep"] = post_eval_ep_diff # stockfish eval if e.p. isn't taken
    return df_row

# decrement_letter("d")
# print(get_advantage(samp.iloc[0]))
samp2 = samp.apply(get_advantage, axis=1)

pre-evaluation: -13
best move: c6e5
post-evaluation: -27
Black
e.p. evaluation: 600
diff: 0
-13

pre-evaluation: -13
best move: c6e5
post-evaluation: -27
Black
e.p. evaluation: 600
diff: 0
-13

pre-evaluation: 200
best move: h2h4
post-evaluation: 223
White
e.p. evaluation: 183
diff: 160
-40

pre-evaluation: -1123
best move: d8d6
post-evaluation: -1168
Black
e.p. evaluation: -1200
diff: -1218
95

pre-evaluation: 138
best move: f7h7
post-evaluation: 23
White
e.p. evaluation: -176
diff: 0
-138

pre-evaluation: -263
best move: f8b4
post-evaluation: -280
Black
e.p. evaluation: 31
diff: 0
-263

pre-evaluation: -412
best move: d5c6
post-evaluation: -422
White
e.p. evaluation: -413
diff: 0
412

pre-evaluation: 9
best move: c5b6
post-evaluation: 25
Black
e.p. evaluation: 42
diff: 0
9

pre-evaluation: -431
best move: b4c3
post-evaluation: -472
Black
e.p. evaluation: -412
diff: -404
-27

pre-evaluation: 933
best move: c4c5
post-evaluation: 5966
White
e.p. evaluation: 5581
diff: 5581
4648

pre-eva

In [120]:
def check_adjacent_piece(square, pawn):
    '''Checks if a piece on a square is an opponent pawn'''
    piece = fish.get_what_is_on_square(square)
    return True if piece == pawn else False

def get_abs_advantage(post, pre, col):
    '''Centipawns is always from white side so account for that'''
    difference = post - pre
    if col=="White":
        return difference
    else:
        return difference * -1
        

def get_best_move_evaluation(pre_eval, col):
    best_move = fish.get_best_move()
    # print("best move: " + str(best_move))
    # make best move and measure
    fish.make_moves_from_current_position([best_move])
    post_eval = fish.get_evaluation()
    # print(post_evaluation["value"])
    
    # print("post-evaluation: " + str(post_eval["value"]))
    
    post_eval_best_diff = get_abs_advantage(post_eval["value"], pre_eval["value"], col)

    # print(post_eval_best_diff)
    # print("----")
    return post_eval_best_diff

def get_adj_squares(board, col, last_move):
    # column in the form "a - g"
    pawn_move_col = last_move[-2]
    # row in the form "1 - 8"
    pawn_move_row = last_move[-1]
    left_sq = decrement_letter(pawn_move_col) + pawn_move_row
    right_sq = increment_letter(pawn_move_col) + pawn_move_row
    return [left_sq, right_sq]
    
def check_valid_pawn(board, col, bounds, squares):
    left_bound, right_bound = bounds
    left_square, right_square = squares
    
    # if colour is white, we are looking for a white pawn
    if col == "White":
        FISH_PAWN = fish.Piece.WHITE_PAWN
    # if colour is black, we are looking for a black pawn
    else:
        FISH_PAWN = fish.Piece.BLACK_PAWN

    # set the fen
    fish.set_fen_position(board.fen())
    # if corresponding square is a pawn of correct colour
    valid_left = False
    valid_right = False
    # if somewhere in the middle then check both sides
    if (not left_bound) and (not right_bound):
        valid_left = check_adjacent_piece(left_square, FISH_PAWN)
        valid_right = check_adjacent_piece(right_square, FISH_PAWN)
    
    # if pawn is on A file (leftmost)
    elif left_bound:
        # check right only
        valid_right = check_adjacent_piece(left_square, FISH_PAWN)
    
    # if pawn is on H file (rightmost)
    elif right_bound:
        # check left only
        valid_left = check_adjacent_piece(right_square, FISH_PAWN)
    
    return valid_left, valid_right

def ep_eval(fen, move, col):
    # reset board to pre-move FEN
    fish.set_fen_position(fen)
    # print(fish.get_board_visual())
    # make the move on stockfish
    fish.make_moves_from_current_position([move])
    
    # print(fish.get_board_visual())
    print("e.p. evaluation: " + str(fish.get_evaluation()["value"]))
    return fish.get_evaluation()["value"]

def get_advantage(df_row):
    check_set = set("abcdefgh")

    
    pgn_file = io.StringIO(df_row["pgn"])   # PGN as a file
    game = chess.pgn.read_game(pgn_file)    # Read PGN and put into game
    board = game.board()                    # "board" of a game
    precheck = False
    
    pre_eval = 0
    post_eval_best_diff = 0
    post_eval_ep_diff = 0
    
    for move in game.mainline_moves():
        san = board.san(move)
        move_piece = san[0]

        # check if precheck was flagged in previous move
        if precheck:
            # colour is the colour of the player with the opportunity
            col = df_row["ep_colour"]
            # print(col)
            
            fish.set_fen_position(board.fen())
            
            # get pre-move evaluation
            pre_eval = fish.get_evaluation()
            # print("pre-evaluation: " + str(pre_eval["value"]))

            post_eval_best_diff = get_best_move_evaluation(pre_eval, col)
            
            
            # e.g. "e7e5"
            last_move = str(board.peek())
            
            # get adjacent squares of the pre-e.p. move
            squares = get_adj_squares(board, col, last_move)
            left_square, right_square = squares
            # checks the move is in leftmost or rightmost column
            left_bound = True if left_square[-2] == "y" else False
            right_bound = True if right_square[-2] == "z" else False
            bounds = [left_bound, right_bound]
            
            ep_left, ep_right = check_valid_pawn(board, col, bounds, squares)
            
            # if white is attacking then e.p. on row 6
            # if black is attacking then e.p. on row 3
            resulting_row = "6" if col == "White" else "3"
            
            eval_ep_left = 0
            eval_ep_right = 0
            
            if ep_left and ep_right:
                move_left = decrement_letter(last_move[-2]) + last_move[-1] + last_move[-2] + resulting_row
                eval_ep_left = ep_eval(board.fen(), move_left, col)
                
                move_right = increment_letter(last_move[-2]) + last_move[-1] + last_move[-2] + resulting_row
                eval_ep_left = ep_eval(board.fen(), move_right, col)

                if col == "White":
                    best_ep_val = max(eval_ep_left, eval_ep_right)
                else:
                    best_ep_val = min(eval_ep_left, eval_ep_right)
                
            if ep_right:
                move = increment_letter(last_move[-2]) + last_move[-1] + last_move[-2] + resulting_row
                best_ep_val = ep_eval(board.fen(), move, col)

            else:
                move = decrement_letter(last_move[-2]) + last_move[-1] + last_move[-2] + resulting_row
                best_ep_val = ep_eval(board.fen(), move, col)
            
            
            print("pre: " + str(pre_eval["value"]))
            if col == "White":
                print("diff: " + str(best_ep_val))
                post_eval_ep_diff = best_ep_val - pre_eval["value"]
            else:
                print("diff: " + str(best_ep_val))
                post_eval_ep_diff = -1*(best_ep_val - pre_eval["value"])
            print()
            break
            
            
        # if the move was a pawn (lower case)
        if move_piece in check_set:
            # Push the move before checking the board
            board.push(move)
            # check if the next move can be en passant
            precheck = board.has_legal_en_passant()
        # if the move wasn't a pawn then just continue as normal
        else:
            board.push(move)

samp2 = samp.apply(get_advantage, axis=1)

e.p. evaluation: 600
pre: -13
diff: 602

e.p. evaluation: 183
pre: 200
diff: 160

e.p. evaluation: -1200
pre: -1123
diff: -1218

e.p. evaluation: -176
pre: 138
diff: -138

e.p. evaluation: 31
pre: -263
diff: 31

e.p. evaluation: -413
pre: -412
diff: -407

e.p. evaluation: 42
pre: 9
diff: 70

e.p. evaluation: -412
pre: -431
diff: -404

e.p. evaluation: 5581
pre: 933
diff: 5581

e.p. evaluation: 170
pre: 212
diff: 189



In [82]:
print(samp2)

      white_rating  black_rating time_class  rated  \
4211          1500          1661      rapid   True   
3488           916           881      rapid   True   
5043           933          1069      blitz   True   
4987          1375          1358     bullet   True   
359            883           805      blitz   True   
4169          1541          1529      blitz   True   
131           1594          1574      rapid   True   
4216          1422          1479      blitz   True   
1545          1006           955      blitz   True   
4946          1438          1462     bullet   True   

                                                    fen  \
4211      6k1/3r1p1p/6p1/pP6/8/P1b4P/3p1PP1/3R2K1 w - -   
3488                  8/8/8/5bQk/3P1Q2/P1P5/8/6K1 b - -   
5043  1rbq1rk1/2p3p1/p1B4p/3p4/4pPn1/3P2Q1/PPP2bPP/R...   
4987        k7/1Q5p/p1P3pP/4P1P1/8/2P5/2n2PK1/4q3 b - -   
359   r1b2rk1/p4p1p/4pPpP/1p1p4/1P2n3/2P1P1P1/5PB1/R...   
4169  1r6/2q5/3p1pk1/3Pn2p/1P2P1pP/2K1QpP1/1N6/5R2 

### Number of games with at least 1 possible en-passant move - ???:
## 4750
(maybe 4913 or even 4945)

### Number of games with an actual en-passant move - ???:
## 1566

#### Applying PCA


In [ ]:
# Replace Boolean and string variables with numbers
chess_filter['ep_happened'] = chess_filter['ep_happened'].replace({True:1, False:0})
chess_filter['ep_colour'] = chess_filter['ep_colour'].replace({'White':1, 'Black':0})
chess_filter['rated'] = chess_filter['rated'].replace({True:1, False:0})
chess_filter['time_class'] = chess_filter['time_class'].replace({'daily':3, 'rapid':2, 'blitz':1, 'bullet':0})

# Drop irrelevant columns, and save differently - as a dataframe including the rating and one without
chess_data_with_elo = chess_filter.drop(['fen', 'pgn', 'move_list'], axis=1)
chess_data_without_elo = chess_data_with_elo.drop(['white_rating', 'black_rating'], axis=1)

print(chess_data_with_elo.head())
print('\n')
print(chess_data_without_elo.head())

: 

##### Data Standardisation (For PCA)

Cite week 8 lab sheet in report? Heavily used their code to help

In [ ]:
# THIS PART IS COPIED AND PASTED FROM WEEK 8 PCA SOLUTIONS:
def sort_eigenvalues(eigenvalues, eigenvectors):
    idx = eigenvalues.argsort()[::-1]   
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:,idx]
    return eigenvalues, eigenvectors

# THE REST IS HEAVILY INFLUENCED BY WEEK 8 PCA SOLUTIONS

# Standardise chess_data_without_elo so that it follows the Standard normal distribution
for col in chess_data_without_elo.columns:
        chess_data_without_elo[col] = (chess_data_without_elo[col] - chess_data_without_elo[col].mean()) / chess_data_without_elo[col].std()

# # Create matrix storing covariances of the chess dataframe features
# covariances = np.cov(chess_data_without_elo.values.T)

# # Record eigenvalues and eigenvectors
# eigenvalues, eigenvectors = np.linalg.eig(covariances)
# eigenvalues, eigenvalues = sort_eigenvalues(eigenvalues, eigenvectors)

# result = chess_data_without_elo.dot(eigenvectors[:,:2])

# sns.scatterplot(x= result[0], y = result[1], hue=diagnosis)
# plt.xlabel('PC1')
# plt.ylabel('PC2')




# Lila Can we instead use this? :
pca = PCA(n_components=3).fit(chess_data_without_elo.values)
pca_result = pca.transform(chess_data_without_elo.values)
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1])
plt.xlabel('PC1')
plt.ylabel('PC2')

: 